## Introducción a plotnine

In [2]:
from plotnine import *

ModuleNotFoundError: No module named 'plotnine'

In [3]:
import pandas as pd

In [4]:
covid = pd.read_csv('datos/covid_19.csv', sep = ';')
covid['Date'] = pd.to_datetime(covid['Date'])

In [5]:
spain = covid[covid['Country/Region'] == 'Spain']
spain = spain.sort_values('Date')
spain = spain[spain.Confirmed > 0]
spain['Day'] = range(1,len(spain) + 1)

### Gráfico de puntos

In [6]:
ggplot(data = spain)

NameError: name 'ggplot' is not defined

In [ ]:
ggplot(data = spain,
           mapping = aes(x = 'Day', y = 'Confirmed'))

In [ ]:
ggplot(data = spain,
           mapping = aes(x = 'Day', y = 'Confirmed')) +  geom_point()

## Gráfico de líneas

In [ ]:
ggplot(data = spain,
           mapping = aes(x = 'Day', y = 'Confirmed')) +  geom_line()

A continuación, pintaremos dos líneas en el mismo gráfico.

In [ ]:
covid_s_i = covid[covid['Country/Region'].isin(['Spain', 'Italy'])]

In [ ]:
g = ggplot(data = covid_s_i,
           mapping = aes(x = 'Date', y = 'Confirmed', color = 'Country/Region')) +  geom_line()

g

In [ ]:
g +  theme(axis_text_x=element_text(rotation=90, hjust=1))


In [ ]:
g +  theme(axis_text_x=element_text(rotation=60, hjust=1))

### Gráfico de barras

Veamos el número total (a nivel mundial) de casos confirmados por mes.

In [ ]:
covid['Month'] = covid.Date.apply(lambda x: x.month)
covid_g = covid.groupby('Month', as_index = False).Confirmed.sum()

In [ ]:
ggplot(data = covid_g,
           mapping = aes(x = 'Month', y = 'Confirmed'))  + geom_col()

In [ ]:
g = ggplot(data = covid_g,
           mapping = aes(x = 'Month', y = 'Confirmed'))  + geom_col(fill = 'lightblue') 

g

Podemos personalizar nuestro gráfico algo más. 

In [ ]:
dodge_text = position_dodge(width=0.9)

(g +  geom_text(aes(label='Confirmed'),                                    
             position=dodge_text,
             size=8, va='bottom')
+ xlab('Mes') 
+ ylab('Total casos confirmados') 
+ theme(panel_background=element_rect(fill='white'),            
         axis_line_x=element_line(color='black'),
         axis_text_y=element_blank(),
         axis_ticks_major_y=element_blank(),
         panel_grid=element_blank(),
         panel_border=element_blank()))

## Gráficos interactivos con Pyplot

In [ ]:
from plotly.tools import mpl_to_plotly

In [ ]:
g = ggplot(data = spain,
           mapping = aes(x = 'Day', y = 'Confirmed')) +  geom_point()

In [ ]:

g = ggplot(data = spain,
           mapping = aes(x = 'Day', y = 'Confirmed')) +  geom_point()

fig = g.draw()

plotly_fig = mpl_to_plotly(fig)
plotly_fig

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(go.Scatter(
                x=spain.groupby('Date')['Date'].first(),
                y=spain.groupby('Date')['Confirmed'].sum(),
                name="Confirmed",
                mode='lines'))


fig.update_layout(title = 'Número de casos activos en España', 
                  xaxis_title="Fecha",
                  yaxis_title="Número de enfermos")

fig.show()

En el siguiente gráfico, podemos ver el número de confirmados frente a fallecidos, en el país o región que seleccionemos

In [ ]:
data = pd.read_csv('covid_19_complete.csv')
data['Date'] = pd.to_datetime(data['Date'],  format='%m/%d/%y')

In [ ]:
region = 'Italy'

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=data[data['Country/Region'] == region].groupby('Date')['Date'].first(),
                y=data[data['Country/Region'] == region].groupby('Date')['Confirmed'].sum(),
                name="Casos confirmados"))

fig.add_trace(go.Scatter(
                x=data[data['Country/Region'] == region].groupby('Date')['Date'].first(),
                y=data[data['Country/Region'] == region].groupby('Date')['Deaths'].sum(),
                name="Muertes"))


fig.update_layout(title="Número de infectados por COVID-19".format(region),
                  xaxis_title="Fecha",
                  yaxis_title="Número de enfermos")

fig.show()

Por último, gracias a las variables longitud y latitud, podemos pintar un mapa.

In [ ]:
data['Province/State'][pd.isnull(data['Province/State'])] = ''

In [ ]:


fig = go.Figure(data=go.Scattergeo(
        lon = data['Long'],
        lat = data['Lat'],
        text = data['Province/State']+ '-' + data['Country/Region'] + ': ' + data['Confirmed'].astype(str),
        mode = 'markers',
       marker_size = 4.5,
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['Confirmed'],
                      cmax = data['Confirmed'].max(),
                      colorbar_title="Confirmed Cases")))

fig.update_layout(title = 'Números de casos activos por País/Estado',
                  geo=dict(scope='world',
                           showland = True,
                           landcolor = "rgb(145, 185, 87)",
                           showocean = True,
                           oceancolor = "rgb(223, 239, 240)",
                           showcountries=True,
                           showsubunits=True,
                           showlakes=False,))
fig.show()